<img src = img/title.png >

# Countdown Numbers Game

***

## Overview and explanation
***

### What is it?


Countdown is a British game show of letters, words and numbers. In the numbers part of the game the players must try to combine six selected numbers using the four elementary mathematical operations to get as close as possilbe to the randomly generated three-digit target number.

### The rules

A contestant starts the numbers game by first picking how many small or large numbers they would like to make up the six numbers. There are 20 small numbers available ranging between 1 and 10 twice and 4 large numbers available, being 25, 50, 75 and 100. All large numbers must be different, so at most 4 large numbers can be chosen. The target number is randomly generated between the range of 101 to 999, the contestants must use basic arithemtic (+, -, × , ÷)on the generated numbers to get as close as possible to the target number, fractions or negative integers are not allowed in any stage of the calculation. You can use as many of the six selected numbers as you want, but can only use each number once.

## Discussion of the complexity
***

The implementation of a Countdown numbers game solver is a problematic and complex challenge. The first thing to discuss is the size of the problem and how many different possible solutions could exist:

In [1]:
import itertools as it
# Number of permutations of the six generated playing numbers.
ops = [1, 2, 5, 10, 75, 100]
nPermutations = 0
for q in it.permutations(ops, 6):
    nPermutations = nPermutations + 1
print(nPermutations) 

720


In [2]:
# Number of combinations of the five operators with replacement.
ops = ['+', '*', '-', '/']
oCombinations = 0
for q in it.product(ops, repeat=5):
    oCombinations = oCombinations+1
print(oCombinations) 

1024


In [3]:
# Number of combinations of RPN
RPN = 42

In [4]:
# Total number of possible solutions to a single generated playing numbers set.
nPermutations * oCombinations * RPN

30965760

Out of the six selected numbers, there are 720 permutations of playing numbers. 1024 different combinations to the five operators that can be used within the calculation. And 42 different ways to format the calculation into a Reverse Polish Notation. In total there are **30,965,760** different ways to write the calculation for the numbers game. However this total only considers one generation set of playing numbers, for every possible set of playing numbers the total is far greater.

In [5]:
# From the Python Standard Library.
import itertools as it

# Find the number of possible generated playing numbers for any game
playingSet = [1,1,2,2,3,3,4,4,5,5,6,6,7,7,8,8,9,9,10,10,25,50,75,100]
generatedSet = 0
for c in it.combinations(playingSet, 6):
    generatedSet = generatedSet + 1
    
print(generatedSet)

134596


In [6]:
# Total number of possible solutions to all possible playing numbers.
generatedSet * oCombinations * RPN

5788704768

Out of all the possible generated sets of playing numbers there are a total of **5,788,704,768** possible solutions. However this can be trimmed significantly as negatives or fractions are not allowed in any stage of the calculation.

## Python function to solve a Countdown numbers game
***

The function should take a list of six numbers and a target
number and return a method to calculate the target from the numbers, if it exists.

## Simulate a game

In [7]:
# For random numbers and samples.
import random

def new_numbers_game(no_large=None):
    """ Returns six numbers and a target number representing a Countdown numbers game.
    """
    # If no_large is None, randomly pick value between 0 and 4 inclusive.
    if no_large is None:
        # Randomly set the value.
        no_large = random.randrange(0, 5)
        
    # Select random large numbers.
    large_rand = random.sample([25, 50, 75, 100], no_large)
    # Select random small numbers.
    small_rand = random.sample(list(range(1, 11)) * 2, 6 - no_large)
    # The playing numbers.
    play_num = large_rand + small_rand
    
        
    # Select a target number.
    target = random.randrange(101, 1000)
    
    # Return the game.
    return play_num, target

In [8]:
# Random numbers game.
play_num, target = new_numbers_game()
play_num, target

([4, 6, 5, 9, 5, 2], 699)

## The solution

In [9]:
# Python itertools
import itertools as it



# Give all 2-partitions of a list 
# where each sublist has one element.
def patterns(numbers, operators):
    # Check if there is no way to partition further.
    if len(numbers) == 1:
        yield numbers
    # Loop through all the ways to partition L into two non-empty sublists
    for i in range(1, len(numbers)):
        # Slice the list using i.
        for left, right in it.product(patterns(numbers[:i], operators[1:i]), patterns(numbers[i:], operators[i:])):
            # Yield the next operator applied to the sublists.
            yield [*left, *right, operators[0]]

In [10]:
# Calculates the RPN expression
def eval_rpn(rpn):
    # A stack.
    stack = []
    # Loop through rpn an item at a time.
    for i in rpn:
        # Check if it's a number.
        if isinstance(i, int):
            # Append to the stack.
            stack = stack + [i]
        else:
            # Pop from stack twice.
            right = stack[-1]
            stack = stack[:-1]
            left = stack[-1]
            stack = stack[:-1]
            # Push operator applied to stack elements.
            stack = stack + [i(left, right)]
            
    # Should only be one item on stack.
    return stack[0]

In [11]:
# Function to convert calculation into operators (not in use)
def calculationConversion(string):
    char_to_replace = {'<built-in function sub>': '-',
                      '<built-in function mul>': '*',
                      '<built-in function add>': '+',
                      '<built-in function truediv>': '/',}
    for key, value in char_to_replace.items():
        # Replace key character with value character in string
        sample_string = sample_string.replace(key, value)
    print(sample_string)

In [86]:
# Python time for measuring calculation speed
import time

# Operators as functions.
import operator

# Declare operators
operators = [operator.add, operator.mul, operator.sub, operator.truediv]

match = False

# Function to calculate the target
def solver(numbers, operators):
    start = time.time()
    
    # Loops through each product of the operators
    for p in it.product(operators, repeat=5):
        
            # Loops through all RPN of each possible operator combination
            for i in patterns(numbers, p):
                # Try catch to solve division by zero exception
                try:
                    j = eval_rpn(i)
                    # Removes negative and fraction calculations
                    if isinstance(eval_rpn(i), int) and eval_rpn(i) > 0:

                        # Checks if the calculation is equal to the target
                        if eval_rpn(i) == target:
                            global match
                            match = True
                            calculation = i                   
                    
                except ZeroDivisionError:
                    j = 0
                     # If target is matched, print stats
    if(match == True):
        print("\nTarget:", target )
        print("Numbers:", numbers)
        print("\nTarget calculation: ",calculation)
        print("\nTook", time.time()-start, "seconds.")
        
                
    if (match != True):
        print("Target:", target)
        print("No calculation found for target")

In [91]:
# Start a new numbers game.
play_num, target = new_numbers_game()
play_num, target

([50, 100, 75, 25, 2, 3], 661)

In [92]:
# Solution calculation for the target
solver(play_num, operators)

Target: 661
No calculation found for target


## Explanation of the functional aspects of the code
***

Functional programming is a programming standard in which everything is wrote in a pure mathematical functional style. It is a declarative paradigm, where the code tells the computer what result it wants, which contrasts an imperative paradigm where programs are written in by composing functions. In a functional program, input flows through a set of functions where each function operatores on its input and produces some form of output. The standard uses expressions instead of statements, an expression is evaluatued to produce a value wheras a statement is executed to assign variables. The functions in this standard are supposed to be closer to the definition of mathematical function, where there are side effects or even no access to external variables. 

Python is not a purely functional programming language, though it does have some of the characteristic concepts. It is possible to use Python in a way that is seen as functional, the functional aspects of this code is explained below:

## References
***

- https://british-game-show.fandom.com/wiki/Countdown
<br>
- https://datagenetics.com/blog/august32014/index.html
<br>
- https://stackoverflow.com/questions/54384059/generating-all-possible-unique-rpn-reverse-polish-notation-expressions/54496061#54496061
<br>
- https://www-stone.ch.cam.ac.uk/documentation/rrf/rpn.html
<br>
- https://docs.python.org/3/howto/functional.html
<br>
- https://towardsdatascience.com/how-to-make-your-python-code-more-functional-b82dad274707
<br>
- https://docs.python.org/3/library/time.html
<br>